In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import PyPDF2

/home/david/grt/grt_reader/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

In [3]:
pdf_path = f"../../data/asc_842/lease_agreements/lease001.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

In [4]:
pdf_text

'EX-10.32\t\n3\t\nex1032hvelease2022.htm\t\nEX-10.32\nExhibit\t10.32\nLEASE\tAGREEMENT\nSTATE\tOF\tTEXAS\t\t\t\t\t\t\t\t§\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t§\nCOUNTY\tOF\tELLIS\t\t\t\t\t\t\t\t§\nThis\tLease\tAgreement\t(“Lease”)\tis\tmade\tand\tentered\tinto\teffective\tas\tof\tthis\t25\n\tday\tof\tJanuary,\t2022\t(“Effective\tDate”)\nby\tand\tbetween\tBarBell\tReal\tEstate,\tLLC,\ta\tTexas\tlimited\tliability\tcompany\t(“Landlord”)\tand\tHVE,\tInc,\ta\tDelaware\nCorporation\t(“Tenant”).\n\t\t\t\tWHEREAS,\tLandlord\tis\tthe\tcurrent\towner\tof\tthe\tLeased\tPremises;\tand\n\t\t\t\tWHEREAS,\tLandlord\tdesires\tto\tlease\tthe\tLeased\tPremises\tto\tTenant,\tand\tTenant\tdesires\tto\tlease\tthe\tLeased\tPremises\nfrom\tLandlord\tsubject\tto\tthe\tterms\tstated\tin\tthis\tLease\tAgreement\tand\tin\tany\tother\tdocuments\tduly\treferenced\tand\nincorporated\therein.\n\t\t\t\tNOW,\tTHEREFORE,\tin\tconsideration\tof\tthe\tpremises\tand\tthe\tmutual\tcovenants\tand\tagreements\tcontained\therei

In [5]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(pdf_text)


/home/david/grt/grt_reader/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
entities_found = 0

for result in ner_results:
    if result['entity'] == 'B-ORG' or result['entity'] == 'I-ORG':
        entities_found += 1
        print(result)

{'entity': 'I-ORG', 'score': 0.42792597, 'index': 44, 'word': '##EX', 'start': 86, 'end': 88}
{'entity': 'B-ORG', 'score': 0.99871266, 'index': 92, 'word': 'Bar', 'start': 279, 'end': 282}
{'entity': 'I-ORG', 'score': 0.9984137, 'index': 93, 'word': '##B', 'start': 282, 'end': 283}
{'entity': 'I-ORG', 'score': 0.9955655, 'index': 94, 'word': '##ell', 'start': 283, 'end': 286}
{'entity': 'I-ORG', 'score': 0.9991903, 'index': 95, 'word': 'Real', 'start': 287, 'end': 291}
{'entity': 'I-ORG', 'score': 0.99911636, 'index': 96, 'word': 'Estate', 'start': 292, 'end': 298}
{'entity': 'I-ORG', 'score': 0.9910499, 'index': 97, 'word': ',', 'start': 298, 'end': 299}
{'entity': 'I-ORG', 'score': 0.97094595, 'index': 98, 'word': 'LLC', 'start': 300, 'end': 303}
{'entity': 'B-ORG', 'score': 0.9711041, 'index': 107, 'word': 'Land', 'start': 341, 'end': 345}
{'entity': 'I-ORG', 'score': 0.9751914, 'index': 108, 'word': '##lord', 'start': 345, 'end': 349}
{'entity': 'I-ORG', 'score': 0.6087705, 'index'

In [7]:
print(f"Found {entities_found} entities in the text")

Found 28 entities in the text
